In [1]:
import ee

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='ee-imhayatkhan')

In [4]:
import geemap
import ee

In [5]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [19]:
m = geemap.Map(center = [53.3811, 1.4701], zoom = 8) # UK
m

Map(center=[53.3811, 1.4701], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [22]:
m = geemap.Map(data_ctrl=False,toolbar_ctrl=False, search_ctrl=False, draw_ctrl=False, measure_ctrl=False, layer_ctrl=False, geocoder_ctrl=False,fullscreen_ctrl=False, home_ctrl=False, add_data_ctrl=False, print_ctrl=False, time_slider_ctrl=False, inspector_ctrl=False, add_marker_ctrl=False, add_polygon_ctrl=False, add_rectangle_ctrl=False, add_circle_ctrl=False, add_ee_layer_ctrl=False, add_text_ctrl=False, add_image_marker_ctrl=False, add_geojson_file_ctrl=False) # UK
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# Adding Basemaps

In [24]:
m = geemap.Map(basemap = 'Esri.WorldImagery')
m.add_basemap('Esri.WorldTopoMap')
m.add_basemap('OpenTopoMap')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [26]:
# Listing Available Basemaps
basemap = list(geemap.basemaps.keys())
print(f'Total basemaps Available: {len(basemap)}')

Total basemaps Available: 150


In [27]:
basemap[0:5] # First 5 Basemaps

['OpenStreetMap',
 'Esri.WorldStreetMap',
 'Esri.WorldImagery',
 'Esri.WorldTopoMap',
 'FWS NWI Wetlands']

# Interactive basemap Select

In [28]:
m = geemap.Map()
m.add('basemap_selector')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…